In [1]:
import numpy as np
import time
import pandas as pd
import os

from selenium import webdriver
import requests
#import platform
#import multiprocessing as mp
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
#from selenium.webdriver.support.select import Select
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Chrome Driver Options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("window-size=2100,700")

In [2]:
# base_url
# url1="https://www.linkedin.com/jobs/digital-marketing-jobs?position=1&pageNum=0"
url1="https://www.linkedin.com/jobs/search/?currentJobId=3711151177&keywords=software%20engineer"

#Load the web driver and get the url
# chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(executable_path=r"C:\\Users\\prave\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver.implicitly_wait(10)
driver.get(url1)

In [3]:
# Get the page source
html_content = driver.page_source

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find the element containing total job listings
total_jobs_element = soup.find("span", {"class": "results-context-header__job-count"})

# Initialize total jobs to 0
total_jobs = 0

# Calculate total jobs if the element is found
if total_jobs_element:
    # Extract numeric value from text
    total_jobs_text = total_jobs_element.get_text().strip().replace(",", "")
    total_jobs_numeric = ''.join(filter(str.isdigit, total_jobs_text))
    total_jobs = int(total_jobs_numeric)

# Calculate the number of pages available (assuming 25 listings per page)
listings_per_page = 25
num_pages = (total_jobs + listings_per_page - 1) // listings_per_page

# Print the total number of pages
print("Total number of jobs:",total_jobs)
print("Total number of pages:", num_pages)

Total number of jobs: 155000
Total number of pages: 6200


In [4]:
i = 1
f=0
h=25

while i<=600 :
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #time.sleep(3)
        #send=driver.find_element_by_xpath("//button[@aria-label='See more jobs']")
        #driver.execute_script("arguments[0].click();", send)    
        buu=driver.find_elements_by_tag_name("button")
        x=[btn for btn in buu if btn.text=="See more jobs"]
        for btn in x:
               driver.execute_script("arguments[0].click();", btn)
                #time.sleep(3)
    
        f+=1
        e=h*f+1
        xpath = f'/html/body/div[1]/div/main/section[2]/ul/li[{e}]/div/a'
        job_link = driver.find_element(By.XPATH, xpath).get_attribute("href")
        print(e)
        print(job_link)
        i+=1
        
    except NoSuchElementException:
        print("No more job links.")
        break 
        
#Find job links and append it to a list
jobList = driver.find_elements_by_class_name('base-card__full-link')
hrefList = []
for e in jobList:
    hrefList.append(e.get_attribute('href'))

    
# jobs_links=pd.DataFrame(hrefList,columns=['links'])
# job_links.to_excel('job_links.xlsx',index=False)

No more job links.


In [5]:
def scrape_job(link):
    # Initialize the Chrome driver
    wd = webdriver.Chrome(options=chrome_options)

    # Navigate to the job link
    wd.get(link)

    # Wait for the page to load
    #time.sleep(5)

    # Get the job information
    job_title = wd.find_element(By.CLASS_NAME, 'topcard__title').text

    try:
        company_name = wd.find_element(By.CLASS_NAME, 'topcard__org-name-link').text
    except:
        company_name = nan

    try:
        location = wd.find_element(By.CLASS_NAME, 'topcard__flavor--bullet').text
    except:
        location = nan
        
    try:
        description = wd.find_element(By.CLASS_NAME, 'show-more-less-html__markup').get_attribute('innerHTML')
        description = str(description)
    except:
        description = np.nan
        
    try:
        seniority_level = wd.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span').text
    except:
        seniority_level = np.nan
        
    try:
        industry=wd.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[4]/span').text
    except:
        industry=np.nan

  
    try:
        job_posting_time = wd.find_element(By.CLASS_NAME, 'posted-time-ago__text').text
    except:
        job_posting_time = np.nan
        
    job_info = {'Job_title': job_title,
                'Location': location,
                'Company_Name': company_name,
                'Job_link': link,
               'Job_description':description,
              'Experience_level':seniority_level,
               'Industry':industry,
               'Job_posting_time':job_posting_time
               }

    # Close the Chrome driver
    wd.quit()

    return job_info


job_info_list = []

for link in hrefList:
    job_info = scrape_job(link)
    job_info_list.append(job_info)

# Create a data frame from the job information
df = pd.DataFrame(job_info_list)

# Save the data frame to a CSV and Excel file
#df.to_csv('job_data.csv', index=False)
# df.to_excel('job_data.xlsx', index=False)

In [7]:
df.head()

,Job_title,Location,Company_Name,Job_link,Job_description,Experience_level,Industry,Job_posting_time
0,Software Engineer - University Grad,"Palo Alto, CA",Glean,https://www.linkedin.com/jobs/view/software-en...,\n <strong>About Glean</strong><p><br><...,Entry level,"Technology, Information and Internet",1 week ago
1,Software Engineer,"San Francisco, CA",Fieldguide,https://www.linkedin.com/jobs/view/software-en...,\n <strong>About us:<br><br></strong>Fi...,Entry level,Software Development,1 month ago
2,Software Engineer (New York City),"New York, NY",January,https://www.linkedin.com/jobs/view/software-en...,"\n At January, we work to ensure that b...",Entry level,"Technology, Information and Internet",2 weeks ago
3,Software Engineer - Early Career,"Sunnyvale, CA",Lockheed Martin,https://www.linkedin.com/jobs/view/software-en...,\n <strong>Description:</strong>By brin...,Entry level,Defense and Space Manufacturing,4 weeks ago
4,Software Engineer Internship (Remote),"Atlanta, GA",The Home Depot,https://www.linkedin.com/jobs/view/software-en...,\n <strong>Internship Overview</strong>...,Internship,Retail,1 week ago
